In [1]:
import os
from pathlib import Path
import fnmatch
import json
import pandas as pd
import re

In [ ]:
for path,dirs,files in os.walk('.\\runs'):
    for file in fnmatch.filter(files,'all_results.json'):
        fullname = os.path.abspath(os.path.join(path,file))
        
        task_name = os.path.basename(path)
        folder_str = os.path.basename(os.path.dirname(os.path.dirname(fullname))) # move up two folders and get folder name
        arguments = {}
        if 'mod' in folder_str:
            arguments['modded'] = True
        else:
            arguments['modded'] = False

        rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", folder_str)
        arguments["num_train_epochs"] = int(rr[1])
        arguments["hidden_dropout"] = float(rr[0])

        # print(arguments)
        with open(os.path.abspath(os.path.join(path,'used_arguments.json')), "w") as f: 
            json.dump(arguments,f)
        

In [7]:
task_names = []
modded_list = []
num_train_epochs_list = []
hidden_dropout_list = []
eval_metrics = []
eval_scores = []



for path,dirs,files in os.walk('.\\runs'):
    for file in fnmatch.filter(files,'used_arguments.json'):
        # fullname = os.path.abspath(os.path.join(path,file))
        task_name = os.path.basename(path)
        with open(os.path.abspath(os.path.join(path,file))) as f:
            d = json.load(f)
            modded = d['modded']
            num_train_epochs = d['num_train_epochs']
            hidden_dropout = d['hidden_dropout']
                       


    for file in fnmatch.filter(files,'all_results.json'):
        fullname = os.path.abspath(os.path.join(path,file))
        task_name = os.path.basename(path)
        with open(fullname) as f:
            d = json.load(f)
            for key,value in d.items():
                task_names.append(task_name)
                modded_list.append(modded)
                hidden_dropout_list.append(hidden_dropout)
                num_train_epochs_list.append(num_train_epochs)
                eval_metrics.append(key)
                eval_scores.append(value)


In [8]:
df = pd.DataFrame({'task' : task_names,
                   'modded' : modded_list,
                       'num_train_epochs' : num_train_epochs_list,
                       'hidden_dropout' : hidden_dropout_list,
                       'metric' : eval_metrics,
                       'score_mod' : eval_scores})#.set_index(['task','metric'])

In [9]:
import dtale

dtale.show(df)